In [55]:
import numpy as np
import pandas as pd
import os
import glob

In [56]:
os.getcwd()

'C:\\Users\\Shivani Bhardwaj\\Downloads\\raw2018'

In [57]:
pd.set_option('display.max_row',120)
pd.set_option('display.max_columns',120)

In [58]:
path = r'C:\\Users\\Shivani Bhardwaj\\Downloads\\raw2018'

In [59]:
columns = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate','Marketing_Airline_Network','Operating_Airline ',\
 'OriginAirportID','Origin','OriginCityName','OriginState','OriginStateName','DestAirportID','Dest','DestCityName',\
 'DestState','DestStateName','CRSDepTime','DepTime','DepDelay','DepDelayMinutes','DepDel15','TaxiOut','TaxiIn',\
 'CRSArrTime','ArrTime','ArrDelay','ArrDelayMinutes','ArrDel15','Cancelled','Diverted','CRSElapsedTime',\
 'ActualElapsedTime','AirTime','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',\
 'DivAirportLandings','DivReachedDest','DivActualElapsedTime','DivArrDelay','DivDistance','Div1Airport',\
 'Div1AirportID','Div2Airport','Div2AirportID']

In [60]:
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f,usecols=columns,dtype={'Div1Airport':'str','Div2Airport':'str'}) for f in all_files), ignore_index=True)

In [62]:
df_copy = df.copy()
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operating_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,TaxiOut,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div2Airport,Div2AirportID
0,2018,1,1,23,2,2018-01-23,DL,9E,10146,ABY,"Albany, GA",GA,Georgia,10397,ATL,"Atlanta, GA",GA,Georgia,1202,1157.0,-5.0,0.0,0.0,14.0,7.0,1304,1256.0,-8.0,0.0,0.0,0.0,0.0,62.0,59.0,38.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,1,24,3,2018-01-24,DL,9E,10146,ABY,"Albany, GA",GA,Georgia,10397,ATL,"Atlanta, GA",GA,Georgia,1202,1157.0,-5.0,0.0,0.0,13.0,12.0,1304,1258.0,-6.0,0.0,0.0,0.0,0.0,62.0,61.0,36.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1,25,4,2018-01-25,DL,9E,10146,ABY,"Albany, GA",GA,Georgia,10397,ATL,"Atlanta, GA",GA,Georgia,1202,1153.0,-9.0,0.0,0.0,18.0,11.0,1304,1302.0,-2.0,0.0,0.0,0.0,0.0,62.0,69.0,40.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1,26,5,2018-01-26,DL,9E,10146,ABY,"Albany, GA",GA,Georgia,10397,ATL,"Atlanta, GA",GA,Georgia,1202,1150.0,-12.0,0.0,0.0,17.0,11.0,1304,1253.0,-11.0,0.0,0.0,0.0,0.0,62.0,63.0,35.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1,27,6,2018-01-27,DL,9E,10146,ABY,"Albany, GA",GA,Georgia,10397,ATL,"Atlanta, GA",GA,Georgia,1400,1355.0,-5.0,0.0,0.0,17.0,11.0,1500,1459.0,-1.0,0.0,0.0,0.0,0.0,60.0,64.0,36.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Treating Missing, Invalid, and Outlier Values

#### 1. Remove Cancelled Flights

In [63]:
#check unique values of various delay indicators when flight was cancelled
print(df[df.Cancelled==1]['CarrierDelay'].unique())
print(df[df.Cancelled==1]['WeatherDelay'].unique())
print(df[df.Cancelled==1]['NASDelay'].unique())
print(df[df.Cancelled==1]['SecurityDelay'].unique())
print(df[df.Cancelled==1]['LateAircraftDelay'].unique())

[nan]
[nan]
[nan]
[nan]
[nan]


In [64]:
#drop cancelled flights as they cannot add value to delay reasons
df.drop(df[df['Cancelled']==1].index,axis=0,inplace=True)

#### 2. DepDelay

In [65]:
# drop flights where departure delay is 0 and departure time is null 
# as the delay cannot be validated with delay reason or  departure time
df.drop(df[(df['DepDelay']==0)&(df['DepTime'].isnull())
          &(df['CarrierDelay'].isnull())&(df['WeatherDelay'].isnull())&(df['NASDelay'].isnull())
          &(df['SecurityDelay'].isnull())&(df['LateAircraftDelay'].isnull())].index,axis=0,inplace=True)

In [66]:
# function to convert time(hhmm) to minutes

def convert_hhmm_to_minutes(val):
    if len(str(val)) in range(1,3):
        return val
    else:
        return (val//100)*60 + val%100

In [67]:
# fill null values of DepDelay
df['DepDelay'].fillna(convert_hhmm_to_minutes(df['DepTime']) - convert_hhmm_to_minutes(df['CRSDepTime']),inplace=True)

### DepDelayMinutes

In [68]:
# fill null values of DepDelayMinutes
df.loc[(df.DepDelayMinutes.isnull()==1) & (df.DepDelay >= 0),'DepDelayMinutes'] = df.DepDelay
df.loc[(df.DepDelayMinutes.isnull()==1) & (df.DepDelay < 0),'DepDelayMinutes'] = 0

### DepDel15

In [69]:
# fill null values of DepDel15
df.loc[(df.DepDel15.isnull()==1) & (df.DepDelay >= 15),'DepDel15'] = 1
df.loc[(df.DepDel15.isnull()==1) & (df.DepDelay < 15),'DepDel15'] = 0

In [70]:
df.shape

(5601119, 49)

In [71]:
df.isnull().sum()

Year                               0
Quarter                            0
Month                              0
DayofMonth                         0
DayOfWeek                          0
FlightDate                         0
Marketing_Airline_Network          0
Operating_Airline                  0
OriginAirportID                    0
Origin                             0
OriginCityName                     0
OriginState                        0
OriginStateName                    0
DestAirportID                      0
Dest                               0
DestCityName                       0
DestState                          0
DestStateName                      0
CRSDepTime                         0
DepTime                            0
DepDelay                           0
DepDelayMinutes                    0
DepDel15                           0
TaxiOut                         6971
TaxiIn                          9166
CRSArrTime                         0
ArrTime                         2231
A

In [86]:
df[df['ArrTime'].isnull()]
          

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operating_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,TaxiOut,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div2Airport,Div2AirportID
1052,2018,1,1,11,4,2018-01-11,DL,9E,10397,ATL,"Atlanta, GA",GA,Georgia,14574,ROA,"Roanoke, VA",VA,Virginia,1552,1548.0,-4.0,0.0,0.0,28.0,NaN,1720,NaN,NaN,NaN,NaN,0.0,1.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,147.0,RIC,14524.0,NaN,NaN
2497,2018,1,1,12,5,2018-01-12,DL,9E,10821,BWI,"Baltimore, MD",MD,Maryland,12478,JFK,"New York, NY",NY,New York,1100,1107.0,7.0,7.0,0.0,29.0,NaN,1218,NaN,NaN,NaN,NaN,0.0,1.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,94.0,PHL,14100.0,NaN,NaN
5933,2018,1,1,22,1,2018-01-22,B6,B6,13204,MCO,"Orlando, FL",FL,Florida,13933,ORH,"Worcester, MA",MA,Massachusetts,1337,1407.0,30.0,30.0,1.0,13.0,NaN,1618,NaN,NaN,NaN,NaN,0.0,1.0,161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,45.0,BOS,10721.0,NaN,NaN
5968,2018,1,1,22,1,2018-01-22,B6,B6,11697,FLL,"Fort Lauderdale, FL",FL,Florida,13933,ORH,"Worcester, MA",MA,Massachusetts,1413,1438.0,25.0,25.0,1.0,18.0,NaN,1713,NaN,NaN,NaN,NaN,0.0,1.0,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,45.0,BOS,10721.0,NaN,NaN
9552,2018,1,1,27,6,2018-01-27,B6,B6,13204,MCO,"Orlando, FL",FL,Florida,10732,BQN,"Aguadilla, PR",PR,Puerto Rico,2138,2148.0,10.0,10.0,0.0,11.0,NaN,118,NaN,NaN,NaN,NaN,0.0,1.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,74.0,SJU,14843.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5650167,2018,3,9,30,7,2018-09-30,AS,KS,10299,ANC,"Anchorage, AK",AK,Alaska,11445,DUT,"Unalaska, AK",AK,Alaska,1335,1625.0,170.0,170.0,1.0,4.0,NaN,1550,NaN,NaN,NaN,NaN,0.0,1.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,792.0,DLG,11336.0,ANC,10299.0
5650169,2018,3,9,30,7,2018-09-30,AS,KS,10299,ANC,"Anchorage, AK",AK,Alaska,11445,DUT,"Unalaska, AK",AK,Alaska,945,1011.0,26.0,26.0,1.0,4.0,NaN,1200,NaN,NaN,NaN,NaN,0.0,1.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,792.0,DLG,11336.0,ANC,10299.0
5659169,2018,3,9,25,2,2018-09-25,DL,DL,13930,ORD,"Chicago, IL",IL,Illinois,12953,LGA,"New York, NY",NY,New York,1930,2347.0,257.0,257.0,1.0,12.0,NaN,2248,NaN,NaN,NaN,NaN,0.0,1.0,138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,11.0,JFK,12478.0,NaN,NaN
5661395,2018,3,9,26,3,2018-09-26,DL,DL,11193,CVG,"Cincinnati, OH",KY,Kentucky,12953,LGA,"New York, NY",NY,New York,1905,2028.0,83.0,83.0,1.0,20.0,NaN,2102,NaN,NaN,NaN,NaN,0.0,1.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,11.0,JFK,12478.0,NaN,NaN


In [87]:
#Dropped the columns where arrival time is not given and no kind of delay is indicated
df.drop(df[(df['ArrTime'].isnull())
          &(df['CarrierDelay'].isnull())&(df['WeatherDelay'].isnull())&(df['NASDelay'].isnull())
          &(df['SecurityDelay'].isnull())&(df['LateAircraftDelay'].isnull())].index,axis=0,inplace=True)

In [88]:
df.shape

(5598888, 49)

In [89]:
df.isnull().sum()

Year                               0
Quarter                            0
Month                              0
DayofMonth                         0
DayOfWeek                          0
FlightDate                         0
Marketing_Airline_Network          0
Operating_Airline                  0
OriginAirportID                    0
Origin                             0
OriginCityName                     0
OriginState                        0
OriginStateName                    0
DestAirportID                      0
Dest                               0
DestCityName                       0
DestState                          0
DestStateName                      0
CRSDepTime                         0
DepTime                            0
DepDelay                           0
DepDelayMinutes                    0
DepDel15                           0
TaxiOut                         6938
TaxiIn                          6935
CRSArrTime                         0
ArrTime                            0
A

In [90]:
# fill null values of ArrDelay
df['ArrDelay'].fillna(convert_hhmm_to_minutes(df['ArrTime']) - convert_hhmm_to_minutes(df['CRSArrTime']),inplace=True)

In [91]:
# fill null values of ArrDelayMinutes
df.loc[(df.ArrDelayMinutes.isnull()==1) & (df.ArrDelay >= 0),'ArrDelayMinutes'] = df.ArrDelay
df.loc[(df.ArrDelayMinutes.isnull()==1) & (df.ArrDelay < 0),'ArrDelayMinutes'] = 0

In [92]:
# fill null values of ArrDel15
df.loc[(df.ArrDel15.isnull()==1) & (df.ArrDelay >= 15),'ArrDel15'] = 1
df.loc[(df.ArrDel15.isnull()==1) & (df.ArrDelay < 15),'ArrDel15'] = 0

In [93]:
df.isnull().sum()

Year                               0
Quarter                            0
Month                              0
DayofMonth                         0
DayOfWeek                          0
FlightDate                         0
Marketing_Airline_Network          0
Operating_Airline                  0
OriginAirportID                    0
Origin                             0
OriginCityName                     0
OriginState                        0
OriginStateName                    0
DestAirportID                      0
Dest                               0
DestCityName                       0
DestState                          0
DestStateName                      0
CRSDepTime                         0
DepTime                            0
DepDelay                           0
DepDelayMinutes                    0
DepDel15                           0
TaxiOut                         6938
TaxiIn                          6935
CRSArrTime                         0
ArrTime                            0
A

In [94]:
df[df['CRSElapsedTime'].isnull()]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operating_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,TaxiOut,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div2Airport,Div2AirportID
2622833,2018,1,2,15,4,2018-02-15,UA,YX,11618,EWR,"Newark, NJ",NJ,New Jersey,11433,DTW,"Detroit, MI",MI,Michigan,825,841.0,16.0,16.0,1.0,56.0,15.0,1035,1753.0,438.0,438.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,552.0,438.0,0.0,CMH,11066.0,NaN,NaN
2622836,2018,1,2,20,2,2018-02-20,UA,YX,11618,EWR,"Newark, NJ",NJ,New Jersey,14492,RDU,"Raleigh/Durham, NC",NC,North Carolina,620,613.0,-7.0,0.0,0.0,25.0,5.0,806,1135.0,209.0,209.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,322.0,209.0,0.0,PIT,14122.0,NaN,NaN
2622837,2018,1,2,21,3,2018-02-21,UA,YX,11624,EYW,"Key West, FL",FL,Florida,11618,EWR,"Newark, NJ",NJ,New Jersey,1635,1640.0,5.0,5.0,0.0,14.0,6.0,1946,2210.0,144.0,144.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,330.0,144.0,0.0,PHL,14100.0,NaN,NaN


In [95]:
#Calculated CRS Elapsed Time for missing values
df['CRSElapsedTime'].fillna(convert_hhmm_to_minutes(df['CRSArrTime']) - convert_hhmm_to_minutes(df['CRSDepTime']),inplace=True)

In [96]:
df.isnull().sum()

Year                               0
Quarter                            0
Month                              0
DayofMonth                         0
DayOfWeek                          0
FlightDate                         0
Marketing_Airline_Network          0
Operating_Airline                  0
OriginAirportID                    0
Origin                             0
OriginCityName                     0
OriginState                        0
OriginStateName                    0
DestAirportID                      0
Dest                               0
DestCityName                       0
DestState                          0
DestStateName                      0
CRSDepTime                         0
DepTime                            0
DepDelay                           0
DepDelayMinutes                    0
DepDel15                           0
TaxiOut                         6938
TaxiIn                          6935
CRSArrTime                         0
ArrTime                            0
A

In [97]:
#Actual Elapse time is null when the flight was diverted
#In this case DivActualElapsedTime is present
df[(df['ActualElapsedTime'].isnull())&(df['DepTime'].notnull())&(df['ArrTime'].notnull())]['Diverted'].unique()

array([1.])

In [99]:
#All the Diverted parameters all null for flights which were not diverted
df[(df['DivReachedDest'].isnull())&(df['DivActualElapsedTime'].isnull())&(df['DivArrDelay'].isnull())&(df['DivDistance'].isnull())&(df['Div1Airport'].isnull())&(df['Div1AirportID'].isnull())&(df['Div2Airport'].isnull())&(df['Div2AirportID'].isnull())]['Diverted'].unique()


array([0.])